In [162]:
import pandas as pd
import geopandas as gpd
from shapely import ops, geometry
import requests
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from datetime import datetime, time, timedelta, date
import statsmodels.api as sm
import math

KEY = "BSeLVQzecQssiPyaI4CBmkFdvMvGarYa" #add personal key here

#### regression2

 - another regression exploratory file

In [58]:
def rename_columns(df, prefix):
    
    for col in df.columns:
        new_col = prefix + col.replace(" ", "")
        df = df.rename(columns={col: new_col})
    return df

In [155]:
road = gpd.read_file('/home/jovyan/work/data/b-roads/road-split/54.gpkg')

slope = pd.read_csv('/home/jovyan/work/data/c-dem/slope/54.csv')
aspect = pd.read_csv('/home/jovyan/work/data/c-dem/aspect/54.csv')
elev = pd.read_csv('/home/jovyan/work/data/c-dem/elevation/54.csv')

slope = rename_columns(slope, 'slope')
aspect = rename_columns(aspect, 'aspect')
elev = rename_columns(elev, 'elev')

data = road.merge(slope, left_index=True, right_index=True)
data = data.merge(aspect, left_index=True, right_index=True)
data = data.merge(elev, left_index=True, right_index=True)

data = data.drop(['elevpixelcount', 'aspectpixelcount', 'slopepixelcount', 'elevZoneID', 'aspectZoneID', 'slopeZoneID'], axis=1)

In [170]:
A = -0.0033
B = 0.2661
C = -0.0768
D = -1.4476
E = 0.0425
F = -0.1774
G = 0.0046
H = 0.4434
I = 0.0056
J = -0.0046
K = 0
L = -0.0002
M = -0.0072
N = 0.0053
O = -0.0018
P = -0.0145
Q = 0.1591
R = -0.0007
S = 0.0195
T = -0.0035
U = -0.1008

In [ ]:
#ALL THE VARIABLES


In [139]:
def call_line(coordinates, fields, timestep):
    #perform API call
    url = 'https://api.tomorrow.io/v4/timelines?apikey=' + KEY
    
    payload = {
        'units':'metric', 
        'timesteps': timestep, 
        #'location': point, 
        'fields': fields, 
        'timezone': 'America/New_York',
        #'startTime': (now - timedelta(days=1)).isoformat(),
        #'endTime': now.isoformat(),
        'location': {
            'type': 'LineString',
            'coordinates': coordinates},
    }
    
    headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
    }
    
    response = requests.request("POST", url, json=payload, headers=headers)
    return response

In [140]:
now = date.today()
response = call_line(res, ['precipitationIntensityAvg'], ['1h'])

In [132]:
(now - timedelta(days=1)).isoformat()

'2021-09-02T03:21:20.954020'

In [133]:
now.isoformat()

'2021-09-03T03:21:20.954020'

In [141]:
response.text

'{"data":{"timelines":[{"timestep":"1h","startTime":"2021-09-02T23:00:00-04:00","endTime":"2021-09-07T11:00:00-04:00","intervals":[{"startTime":"2021-09-02T23:00:00-04:00","values":{"precipitationIntensityAvg":0}},{"startTime":"2021-09-03T00:00:00-04:00","values":{"precipitationIntensityAvg":0}},{"startTime":"2021-09-03T01:00:00-04:00","values":{"precipitationIntensityAvg":0}},{"startTime":"2021-09-03T02:00:00-04:00","values":{"precipitationIntensityAvg":0}},{"startTime":"2021-09-03T03:00:00-04:00","values":{"precipitationIntensityAvg":0}},{"startTime":"2021-09-03T04:00:00-04:00","values":{"precipitationIntensityAvg":0.0652}},{"startTime":"2021-09-03T05:00:00-04:00","values":{"precipitationIntensityAvg":0.6935}},{"startTime":"2021-09-03T06:00:00-04:00","values":{"precipitationIntensityAvg":0.2146}},{"startTime":"2021-09-03T07:00:00-04:00","values":{"precipitationIntensityAvg":0.3021}},{"startTime":"2021-09-03T08:00:00-04:00","values":{"precipitationIntensityAvg":0}},{"startTime":"2021-

In [120]:
def format_response(response):
    text = json.loads(response.text)
    timelines = text['data']['timelines']
    nest = timelines[0]['intervals'] #list - each element is a dictionary with startTime and values
    startTimes = pd.DataFrame(nest)['startTime']

    val = nest[0]['values'] #first time-stamp value
    values = pd.DataFrame(val, index=[0])

    #iterate through all time-stamps
    for i in range(1, len(startTimes)):
        val = nest[i]['values']
        df = pd.DataFrame(val, index=[i])
        values = pd.concat([values, df], axis=0)

    data = pd.concat([startTimes, values], axis=1)
    return data

In [143]:
table = format_response(response)

In [147]:
table.head(24).sum()['precipitationIntensityAvg']

1.3868999999999998

In [179]:
for index, row in data.head(10).iterrows():
    geom = row['geometry']
    geom_list = list(ops.linemerge(geom).coords)
    res = [list(ele) for ele in geom_list]
    response = call_line(res, ['precipitationIntensityAvg'], ['1h'])
    table = format_response(response)
    data.loc[index, 'precip_1d'] = table.head(23).sum()['precipitationIntensityAvg']
    prob = 1 / (1 + math.exp(-(A*row['precip_1d'] + B*row['slopeMean'] + C*row['slopeMedian'] + D*row['slopeStandarddeviation'] + E*row['slopeVariance'] \
                               + F*row['slopeMinimum'] + G*row['slopeMaximum'] + H*row['slopeMedianabsolutedeviation'] \
                               + I*row['aspectMedian'] + J*row['aspectStandarddeviation'] + K*row['aspectVariance'] \
                               + L*row['aspectMinimum'] +  M*row['aspectMaximum'] + N*row['aspectMedianabsolutedeviation'] \
                               + O*row['elevMean'] + P*row['elevMedian'] + Q*row['elevStandarddeviation'] \
                               + R*row['elevVariance'] + S*row['elevMinimum'] + T*row['elevMaximum'] + U*row['elevMedianabsolutedeviation']
                              )))
    data.at[index, 'probability'] = prob

In [181]:
data.head(10)

,road_id,date,status,segment_id,geometry,slopeMean,slopeMedian,slopeStandarddeviation,slopeVariance,slopeMinimum,...,aspectMedianabsolutedeviation,elevMean,elevMedian,elevStandarddeviation,elevVariance,elevMinimum,elevMaximum,elevMedianabsolutedeviation,precip_1d,probability
0,54,2021-07-12,0,1,"MULTILINESTRING ((79.21719 30.26079, 79.21749 ...",23.638893,24.319117,8.474287,71.813545,1.284740,...,10.738800,808.00000,796.0,42.510406,1807.13460,754.0,916.0,30.5,1.7620,0.022743
1,54,2021-07-12,0,2,"MULTILINESTRING ((79.21827 30.25553, 79.21801 ...",23.578651,22.134052,6.958408,48.419437,11.326518,...,6.795094,861.46155,863.5,40.868988,1670.27430,764.0,970.0,24.5,1.7620,0.053874
2,54,2021-07-12,0,3,"MULTILINESTRING ((79.21520 30.25862, 79.21484 ...",25.205584,25.663105,6.730309,45.297054,11.326518,...,9.615334,922.18750,913.5,56.122880,3149.77730,821.0,1052.0,40.5,1.7620,0.051355
3,54,2021-07-12,0,4,"MULTILINESTRING ((79.21620 30.25347, 79.21600 ...",31.331505,32.356514,8.160066,66.586680,1.126835,...,17.704430,984.60360,982.5,48.545082,2356.62500,876.0,1107.0,36.5,9.5271,0.024116
4,54,2021-07-12,0,5,"MULTILINESTRING ((79.21417 30.24601, 79.21422 ...",26.850680,25.347145,9.144037,83.613420,4.636514,...,81.287388,1019.46466,1017.0,42.133940,1775.26890,929.0,1108.0,35.0,17.2923,0.006063
5,54,2021-07-12,0,6,"MULTILINESTRING ((79.21905 30.24550, 79.21929 ...",25.903196,25.727700,6.961668,48.464825,9.404054,...,16.007416,1089.11110,1088.0,31.572598,996.82886,1034.0,1145.0,27.0,17.2923,0.023855
6,54,2021-07-12,0,7,"MULTILINESTRING ((79.21855 30.24397, 79.21830 ...",26.198477,26.604406,5.853899,34.268135,11.931026,...,61.620454,1110.96910,1111.0,39.823750,1585.93120,1034.0,1200.0,32.0,17.2923,0.005210
7,54,2021-07-12,0,8,"MULTILINESTRING ((79.22114 30.24374, 79.22138 ...",24.976776,24.717830,7.458657,55.631570,10.153459,...,7.758698,1161.93330,1154.0,38.753788,1501.85620,1088.0,1255.0,26.0,17.2923,0.019883
8,54,2021-07-12,0,9,"MULTILINESTRING ((79.22994 30.24412, 79.22943 ...",27.485003,28.297344,7.206075,51.927517,10.153459,...,10.275452,1225.74060,1222.0,63.923412,4086.20300,1103.0,1374.0,49.0,17.2923,0.010209
9,54,2021-07-12,0,10,"MULTILINESTRING ((79.22076 30.24131, 79.22040 ...",27.064394,27.587190,6.578073,43.271040,3.978074,...,46.851930,1296.72680,1293.0,45.554752,2075.23560,1209.0,1407.0,31.5,17.2923,0.020472
